In [1]:
import pandas as pd
import numpy as np

c:\Users\joshu\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\joshu\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
# constants
SEED = 2020222
NO_OF_SIMULATIONS = 1000
FX_CHARGES_RATE = 0.01

In [3]:
# configs
np.random.seed(SEED)

In [4]:
import pandas as pd
import numpy as np

In [5]:
# BoE data from 2020 Oct to 2025 Oct
boe_raw = pd.read_csv('bank_of_england_data.csv')
boe_raw.columns = ['date', 'gbp_to_chf']
boe_raw.head()

,date,gbp_to_chf
0,30 Oct 25,1.0544
1,29 Oct 25,1.0537
2,28 Oct 25,1.0526
3,27 Oct 25,1.0617
4,24 Oct 25,1.0578


In [6]:
# Convert date col to datetime
boe_raw['date'] = pd.to_datetime(boe_raw['date'], format='%d %b %y')

# Add date + month cols
boe_raw['year'] = boe_raw['date'].dt.year
boe_raw['month'] = boe_raw['date'].dt.month

In [7]:
def simulate_once_a_month_conversion(df, seed=101):
    # select a random date for each month, assume we do conenversion calculations on this date 
    random_dates_df = (
        df.groupby(['year', 'month'])[['date', 'gbp_to_chf', 'year', 'month']]
        .apply(lambda x: x.sample(1, random_state=seed))
        .reset_index(drop=True)
        .sort_values('date')
    )
    return random_dates_df.sort_values('date')

In [8]:
def simulate_fx_investment(df, monthly_gbp, fee_rate=0.02):
    df = df.sort_values('date').reset_index(drop=True)

    df['gbp_invested'] = monthly_gbp
    # apply FX conversion fee rate 
    df['gbp_after_fee'] = monthly_gbp * (1 - fee_rate)
    df['chf_bought'] = df['gbp_after_fee'] * df['gbp_to_chf']

    total_chf = df['chf_bought'].sum()
    latest_rate = df['gbp_to_chf'].iloc[-1]
    final_value_gbp = total_chf / latest_rate
    return total_chf, final_value_gbp, df

In [9]:
conversion_dates_df = simulate_once_a_month_conversion(boe_raw, seed=10)

In [10]:
def compare_fx_investment(conversion_dates_df, monthly_investment, fee_rate, print_summary=False):
    total_chf, final_value_gbp, summary = simulate_fx_investment(
        conversion_dates_df, monthly_gbp=monthly_investment, fee_rate=fee_rate
    )

    # GBP total if never converted
    unconverted_gbp = monthly_investment * len(conversion_dates_df)

    # Difference between converted and unconverted
    difference = final_value_gbp - unconverted_gbp
    percent_diff = (difference / unconverted_gbp) * 100

    # Print a quick summary
    if print_summary:
        print(f"Present value of CHF holdings (in GBP): {final_value_gbp:.2f}")
        print(f"Total GBP without conversion: {unconverted_gbp:.2f}")
        print(f"Diff: {difference:.2f}")
        print(f"% diff: {percent_diff:.2f}")

    return {
        "present_value_chf_in_gbp": final_value_gbp,
        "unconverted_gbp": unconverted_gbp,
        "diff": difference,
        "%diff": percent_diff,
        "summary": summary
    }

res = compare_fx_investment(conversion_dates_df, monthly_investment=200, fee_rate=FX_CHARGES_RATE, print_summary=True)

Present value of CHF holdings (in GBP): 12856.98
Total GBP without conversion: 12000.00
Diff: 856.98
% diff: 7.14


In [11]:
seeds = np.random.randint(1, 1_000_000, size=NO_OF_SIMULATIONS).tolist()

In [12]:
def simulate_avg_percent_diff(
    boe_raw,
    start_year,
    start_month, 
    end_year,
    end_month,
    seeds,
    monthly_investment=10000,
    fee_rate=0.02
):
    start_date = pd.Timestamp(year=start_year, month=start_month, day=1)
    end_date = pd.Timestamp(year=end_year, month=end_month, day=1) + pd.offsets.MonthEnd(1)
    date_filtered = boe_raw[(boe_raw['date'] >= start_date) & (boe_raw['date'] <= end_date)]

    percent_diffs = []

    for seed in seeds:
        conversion_dates_df = simulate_once_a_month_conversion(df=date_filtered, seed=seed)
        res = compare_fx_investment(
            conversion_dates_df,
            monthly_investment=monthly_investment,
            fee_rate=fee_rate
        )
        percent_diffs.append(res['%diff'])

    avg_percent_diff = sum(percent_diffs) / len(percent_diffs)
    print(
        f"Ave % diff over {len(seeds)} simulations "
        f"from {start_date.strftime('%b %Y')} to {end_date.strftime('%b %Y')}: {avg_percent_diff:.2f}%"
    )

    return avg_percent_diff


In [13]:
print('1yr case')
avg_diff = simulate_avg_percent_diff(
    boe_raw=boe_raw,
    start_year=2023,
    start_month=10,
    end_year=2025,
    end_month=11,
    seeds=range(100),
    monthly_investment=1000,
    fee_rate=FX_CHARGES_RATE
)

1yr case
Ave % diff over 100 simulations from Oct 2023 to Nov 2025: 3.33%


In [14]:
print('2yr case')
avg_diff = simulate_avg_percent_diff(
    boe_raw=boe_raw,
    start_year=2024,
    start_month=10,
    end_year=2025,
    end_month=11,
    seeds=range(100),
    monthly_investment=1000,
    fee_rate=FX_CHARGES_RATE
)

2yr case
Ave % diff over 100 simulations from Oct 2024 to Nov 2025: 2.78%


In [15]:
print('5yr case')
avg_diff = simulate_avg_percent_diff(
    boe_raw=boe_raw,
    start_year=2020,
    start_month=10,
    end_year=2025,
    end_month=11,
    seeds=range(100),
    monthly_investment=1000,
    fee_rate=FX_CHARGES_RATE
)

5yr case
Ave % diff over 100 simulations from Oct 2020 to Nov 2025: 7.67%


In [ ]:
import plotly.graph_objects as go

def rolling_avg_diff_plotly(
    boe_raw,
    seeds,
    start_year,
    start_month,
    end_year,
    end_month,
    monthly_investment,
    fee_rate,
    window_months
):
    start_date = pd.Timestamp(year=start_year, month=start_month, day=1)
    end_date = pd.Timestamp(year=end_year, month=end_month, day=1)
    all_months = pd.date_range(start=start_date, end=end_date, freq='MS')

    results = []

    for start in all_months:
        end = start + pd.DateOffset(months=window_months) - pd.DateOffset(days=1)
        if end > end_date:
            break

        avg_diff = simulate_avg_percent_diff(
            boe_raw=boe_raw,
            start_year=start.year,
            start_month=start.month,
            end_year=end.year,
            end_month=end.month,
            seeds=seeds,
            monthly_investment=monthly_investment,
            fee_rate=fee_rate
        )
        results.append({"start": start, "avg_diff": avg_diff})

    results_df = pd.DataFrame(results)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=results_df["start"],
        y=results_df["avg_diff"],
        mode='lines+markers',
        line=dict(width=2, color='royalblue'),
        marker=dict(size=6),
        name='Avg % Diff',
        hovertemplate='%{x|%b %Y}<br>%{y:.2f}%<extra></extra>'
    ))

    fig.update_layout(
        title=f"Rolling window of ave in % Difference in FX Investment",
        xaxis_title="Window Start Date",
        yaxis_title="Ave % Difference",
        hovermode="x unified",
        template="plotly_white",
        font=dict(size=14),
        width=900,
        height=500
    )

    fig.show()
    return results_df

In [ ]:
print('Rolling avg diff - 12 months window')
results_df = rolling_avg_diff_plotly(
    boe_raw=boe_raw,
    seeds=range(NO_OF_SIMULATIONS),
    start_year=2020,
    start_month=10,
    end_year=2025,
    end_month=11,
    monthly_investment=1000,
    fee_rate=FX_CHARGES_RATE,
    window_months=12
)

Rolling avg diff - 12 months window
Ave % diff over 100 simulations from Oct 2020 to Sep 2021: -2.20%
Ave % diff over 100 simulations from Nov 2020 to Oct 2021: -1.92%
Ave % diff over 100 simulations from Dec 2020 to Nov 2021: 0.35%
Ave % diff over 100 simulations from Jan 2021 to Dec 2021: 1.60%
Ave % diff over 100 simulations from Feb 2021 to Jan 2022: 0.06%
Ave % diff over 100 simulations from Mar 2021 to Feb 2022: -0.09%
Ave % diff over 100 simulations from Apr 2021 to Mar 2022: 1.53%
Ave % diff over 100 simulations from May 2021 to Apr 2022: 1.41%
Ave % diff over 100 simulations from Jun 2021 to May 2022: 1.25%
Ave % diff over 100 simulations from Jul 2021 to Jun 2022: 3.05%
Ave % diff over 100 simulations from Aug 2021 to Jul 2022: 4.79%
Ave % diff over 100 simulations from Sep 2021 to Aug 2022: 5.33%
Ave % diff over 100 simulations from Oct 2021 to Sep 2022: 8.29%
Ave % diff over 100 simulations from Nov 2021 to Oct 2022: 5.19%
Ave % diff over 100 simulations from Dec 2021 to No

c:\Users\joshu\anaconda3\lib\site-packages\_plotly_utils\basevalidators.py:107: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:
print('Rolling avg diff - 24 months window')
results_df = rolling_avg_diff_plotly(
    boe_raw=boe_raw,
    seeds=range(NO_OF_SIMULATIONS),
    start_year=2020,
    start_month=10,
    end_year=2025,
    end_month=11,
    monthly_investment=1000,
    fee_rate=FX_CHARGES_RATE,
    window_months=24
)

Rolling avg diff - 24 months window
Ave % diff over 100 simulations from Oct 2020 to Sep 2022: 10.14%
Ave % diff over 100 simulations from Nov 2020 to Oct 2022: 7.63%
Ave % diff over 100 simulations from Dec 2020 to Nov 2022: 6.76%
Ave % diff over 100 simulations from Jan 2021 to Dec 2022: 6.39%
Ave % diff over 100 simulations from Feb 2021 to Jan 2023: 6.38%
Ave % diff over 100 simulations from Mar 2021 to Feb 2023: 7.11%
Ave % diff over 100 simulations from Apr 2021 to Mar 2023: 6.25%
Ave % diff over 100 simulations from May 2021 to Apr 2023: 6.01%
Ave % diff over 100 simulations from Jun 2021 to May 2023: 5.13%
Ave % diff over 100 simulations from Jul 2021 to Jun 2023: 3.21%
Ave % diff over 100 simulations from Aug 2021 to Jul 2023: 3.68%
Ave % diff over 100 simulations from Sep 2021 to Aug 2023: 4.03%
Ave % diff over 100 simulations from Oct 2021 to Sep 2023: 3.63%
Ave % diff over 100 simulations from Nov 2021 to Oct 2023: 4.12%
Ave % diff over 100 simulations from Dec 2021 to Nov 

c:\Users\joshu\anaconda3\lib\site-packages\_plotly_utils\basevalidators.py:107: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [19]:
print('Rolling avg diff - 36 months window')
results_df = rolling_avg_diff_plotly(
    boe_raw=boe_raw,
    seeds=range(NO_OF_SIMULATIONS),
    start_year=2020,
    start_month=10,
    end_year=2025,
    end_month=11,
    monthly_investment=1000,
    fee_rate=FX_CHARGES_RATE,
    window_months=36
)

Rolling avg diff - 36 months window
Ave % diff over 100 simulations from Oct 2020 to Sep 2023: 5.99%
Ave % diff over 100 simulations from Nov 2020 to Oct 2023: 6.89%
Ave % diff over 100 simulations from Dec 2020 to Nov 2023: 6.36%
Ave % diff over 100 simulations from Jan 2021 to Dec 2023: 7.20%
Ave % diff over 100 simulations from Feb 2021 to Jan 2024: 7.37%
Ave % diff over 100 simulations from Mar 2021 to Feb 2024: 5.21%
Ave % diff over 100 simulations from Apr 2021 to Mar 2024: 2.79%
Ave % diff over 100 simulations from May 2021 to Apr 2024: 1.59%
Ave % diff over 100 simulations from Jun 2021 to May 2024: 0.61%
Ave % diff over 100 simulations from Jul 2021 to Jun 2024: 1.32%
Ave % diff over 100 simulations from Aug 2021 to Jul 2024: -0.11%
Ave % diff over 100 simulations from Sep 2021 to Aug 2024: 3.00%
Ave % diff over 100 simulations from Oct 2021 to Sep 2024: 1.72%
Ave % diff over 100 simulations from Nov 2021 to Oct 2024: 1.06%
Ave % diff over 100 simulations from Dec 2021 to Nov 

c:\Users\joshu\anaconda3\lib\site-packages\_plotly_utils\basevalidators.py:107: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

